<a href="https://www.kaggle.com/code/emdogan/945-deberta-3-base-striding-inference-tercume?scriptVersionId=166929059" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>


"""## 🛑 Wait a second - maybe also look at the training
- My training notebook (containing equally many emojis) is here: I would love an upvote if you use the notebook or learned something new!
- https://www.kaggle.com/code/valentinwerner/915-deberta3base-training
"""

**Training notebook linkini eklemiş**

## 🏟️ Credits (because this baseline did mostly already exist when I joined)

- Stride is something Raja first shared: https://www.kaggle.com/competitions/pii-detection-removal-from-educational-data/discussion/473011
- The biggest booster in performance is changing triplets to pairs in token reassembling: https://www.kaggle.com/code/nbroad/transformer-ner-baseline-lb-0-854/comments#2659393

<div style="background-color:purple; color:white; padding:10px; border-radius: 5px; text-align:center;">
  <h1>Kod blok no: 1</h1>
</div>

In [ ]:
# dosya yolundan modelin yolunu bulmak için listeleme
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
INFERENCE_MAX_LENGTH=1024
STRIDE=384

# Note that training a model with stride, such as: https://www.kaggle.com/code/thedrcat/pii-data-detection-train-with-w-b
# will also improve performance
model_path = "/kaggle/input/915-deberta3base-training/deberta3base_1024/"

"""
---------------------------------
- modelde kullanılmak üzere değişkenler ve değerler belirlenmiş.

+ maximum giriş uzunluğu yani "INFERENCE_MAX_LENGTH" 1024 karakter olarak belirlenmiş
+ modelin üzerindeki adım büyüklüğümüz yani "STRIDE" değeri 384 olarak belirlenmiş
+ "model_path" değişkenine deberta3 modelinin yolu atanmış
+ not olarak stride ile çalışmanın performansa katkısı olduğu belirtilmiş
---------------------------------
"""

In [ ]:
import json
import argparse
from itertools import chain
import pandas as pd
from pathlib import Path
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments, DataCollatorForTokenClassification
from datasets import Dataset 
import numpy as np

## ♟️ Data Loading & Data Tokenization
- This tokenizer is actually special, comparing to usual NLP challenges
- inference tokenizer is a bit different than training tokenizer, because we don't have labels

In [ ]:
def tokenize(example, tokenizer):
    text = []
    token_map = []
    
    idx = 0
    
    for t, ws in zip(example["tokens"], example["trailing_whitespace"]):
        text.append(t)
        token_map.extend([idx]*len(t))
        if ws:
            text.append(" ")
            token_map.append(-1)
            
        idx += 1
        
    tokenized = tokenizer("".join(text), return_offsets_mapping=True, truncation=True, max_length=INFERENCE_MAX_LENGTH, stride=STRIDE, return_overflowing_tokens=True)
        
    return {
        **tokenized,
        "token_map": token_map,
    }

"""
---------------------------------
- tokenize fonksiyonu oluşturulmuş, (bir önceki incelenen model ile aynı mantıkta çalışıyor) metni alıp tokenize edip bize bir 
    sözlük döndürüyor,

+ fonksiyon içindeki her döngüde label "text" listesine eklenir, "token_map" değişkenine de labelin orjinal metindeki konumu atanır
+ return olarak döndürülen sözlükte tokenized yani tokenize edilmiş metnimiz ve token_map değişkeni yani tokenlerin metindeki konumlarını 
    döndürmüş olur
---------------------------------
"""

In [ ]:
####
data = json.load(open("/kaggle/input/pii-detection-removal-from-educational-data/test.json"))

ds = Dataset.from_dict({
    "full_text": [x["full_text"] for x in data],
    "document": [x["document"] for x in data],
    "tokens": [x["tokens"] for x in data],
    "trailing_whitespace": [x["trailing_whitespace"] for x in data],
})

tokenizer = AutoTokenizer.from_pretrained(model_path)
ds = ds.map(tokenize, fn_kwargs={"tokenizer": tokenizer}, num_proc=2)

"""
---------------------------------
- Test dosyası json formatında olduğu için ".load" ile "data" değişkenine yüklenmiş ardından sözlük haline getirilip "ds" değişkenine 
    atanmış  ve "ds" değişkenine tokenizer uygulanmış.
    + Yani artık test setimiz sözlük biçiminde ve tokenize edilmiş olarak "ds" değişkeninde.
  
+ önceden eğitilmiş tokenizer modelini "tokenizer" değişkenine atamış.
+ map fonksiyonu ile ds değişkenindeki her değişkene tokenize uygulanmış ve kullanılcak olan işlemci çekirdeği 2 olarak belirlenmiş
---------------------------------
"""

"""
# datasetin ekrana yazdırılmış hali
import pandas as pd
ds_df = ds.to_pandas()
print(ds_df)
"""

## 🏋🏻‍♀️ Trainer Class based on the trained model

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(model_path)
collator = DataCollatorForTokenClassification(tokenizer, pad_to_multiple_of=16)
args = TrainingArguments(
    ".", 
    per_device_eval_batch_size=1, 
    report_to="none",
)
trainer = Trainer(
    model=model, 
    args=args, 
    data_collator=collator, 
    tokenizer=tokenizer,
)

"""
---------------------------------
- eğitimde kullanılması için model kolektör argüman değerleri ve train değerlerine atama yapılmış

+ daha önce eğitilmiş model kullanılarak "model", "collator" değişkenlerine atama yapılmış
+ eğitim argümanları "args" değişkenine tanımlanmış
  + "." modelin eğitim sırasında kaydedilceği dizini temsil ediyor yani current location
  + eğitimde her bir GPU veya TPU yani kullanılacak device için, kullanılcak örneklerin sayısı 1 olarak belirlenmiş
  + eğitim sırasında günlüklerin logların nereye kaydedilceği None olarak belirlenmiş yani log kaydı tutulmayacak
+ eğitim sırasında kullanılcak model, argüman, kollektör ve tokenizer ataması trainer nesnesi içinde atanmış
---------------------------------
"""

<div style="background-color:purple; color:white; padding:10px; border-radius: 5px; text-align:center;">
  <h1>Kod blok no: 2</h1>
</div>

### Striding functions

As using the stride give an overlap in tokens, these have to be removed (either pick one side of the stride or average them, ...).

In [ ]:
def backwards_map_preds(sub_predictions, max_len):
    if max_len != 1: # nothing to map backwards if sequence is too short to be split in the first place
        if i == 0:
            # First sequence needs no SEP token (used to end a sequence)
            sub_predictions = sub_predictions[:,:-1,:]
        elif i == max_len-1:
            # End sequence needs to CLS token + Stride tokens 
            sub_predictions = sub_predictions[:,1+STRIDE:,:] # CLS tokens + Stride tokens
        else:
            # Middle sequence needs to CLS token + Stride tokens + SEP token
            sub_predictions = sub_predictions[:,1+STRIDE:-1,:]
    return sub_predictions

"""
---------------------------------
- Alt tahminlerin düzeltilmesi ve düzenlenmesine yardımcı olur 

+ alt tahmin: Büyük metin parçalarını daha küçük parçalara bölerken ve bu parçalar üzerinde tahmin yaparken oluşur
+ SEP (Separation): metinler arasında ayrım yapmak için kullanılan belirteçtir
+ CLS (Classification): metnin genel temsilini yakalamayı sağlar, sınıflandırma görevlerinde modelin metni sınıf veya kategoriye 
    ayırmasına yardımcı olur

+ "max_len" 1'e eşit değilse metin parçasının uzun olduğunu ve bölünmesi gerektiği anlamına gelir ve if içine girerek çalışır
+ i eğer 0'a eşitse yani alt dizinin başındaysak (ilk alt dize ise) bu koşula girer ve buraya eklenen belirteçlerin (SEP) tahminlerden 
    kaldırılmasını sağlar
+ i eğer max_len-1 ise yani son alt dizi ise bu koşula girilir, eklenen belirteçlerin (CLS) tahminlerden kaldırılmasını sağlar
+ diğer iki koşul değilse else durumuna girer ve bu da dizinin orta bloğunda olduğumuzu temsil eder bu durumda da eklenen belirteçlerin 
    (SEP, CLS) tahminlerden kaldırılması sağlanır
---------------------------------
"""

def backwards_map_(row_attribute, max_len):
    # Same logics as for backwards_map_preds - except lists instead of 3darray
    if max_len != 1:
        if i == 0:
            row_attribute = row_attribute[:-1]
        elif i == max_len-1:
            row_attribute = row_attribute[1+STRIDE:]
        else:
            row_attribute = row_attribute[1+STRIDE:-1]
    return row_attribute

"""
---------------------------------
- Tahminlerin yanı sıra attributes'ların da düzenlenmesine yardımcı olur, backwards_map_preds ile benzerdir ama Preds yani tahminler 
    yerine öznitekiklerin düzenlenmesine odaklanır
---------------------------------
"""

In [ ]:
%%time
preds = []
ds_dict = {
    "document":[],
    "token_map":[],
    "offset_mapping":[],
    "tokens":[]
}

for row in ds:
    # keys that need to be re-assembled
    row_preds = []
    row_offset = []

    for i, y in enumerate(row["offset_mapping"]):
        # create new datasset for each of of the splits per document
        x = Dataset.from_dict({
            "token_type_ids":[row["token_type_ids"][i]],
            "input_ids":[row["input_ids"][i]],
            "attention_mask":[row["attention_mask"][i]],
            "offset_mapping":[row["offset_mapping"][i]]
        })
        # predict for that split
        pred = trainer.predict(x).predictions
        # removing the stride and additional CLS & SEP that are created
        row_preds.append(backwards_map_preds(pred, len(row["offset_mapping"])))
        row_offset += backwards_map_(y, len(row["offset_mapping"]))
    
    # Finalize row
    ds_dict["document"].append(row["document"])
    ds_dict["tokens"].append(row["tokens"])
    ds_dict["token_map"].append(row["token_map"])
    ds_dict["offset_mapping"].append(row_offset)
    
    # Finalize prediction collection by concattenating
    p_concat = np.concatenate(row_preds, axis = 1)
    preds.append(p_concat)
"""
---------------------------------
- Bu blokta Datasetimizdeki her bir row için döngü başlatılmış ve önceden eğitilmiş modelimiz ile tahminleme yapılmış, ardından bu 
    tahminleri düzenleyip "preds" listesine eklemiş, ayrıca her bir rowun giriş özellikleri ve diğer verileri "ds_dict" sözlüğüne eklenmiş. 

+ %%time kullanarak kod bloğunun çalışma süresi ölçülmüş
+ "preds" ve "ds_dict" kod bloğunun başında empty olarak hazır hale getirilmiş
+ döngü başlatılmış ve her bir parça için giriş bilgilerini içeren ds oluşturulmuş ve x değişkenine atanmış
  + token türü kimliği, giriş idleri, attention mask ve offset map bilgilerini içerir
+ trainer fonksiyonu ile bu "x" değişkeni kullanılarak tahminler yapılmış
+ "backwards_map_preds" ve "backwards_map_" kullanılarak tahmin ve offsetler düzenlenmiş
  + tahminlerin ve offset eşleşmelerinin stride yani bölen metinler için düzenlenmesi sağlanmış
+ "ds_dict" sözlüğüne "document", "tokens", "token_map" ve "offset_mapping" eklenmiş

---------------------------------
"""

In [ ]:
config = json.load(open(Path(model_path) / "config.json"))
id2label = config["id2label"]

preds_final = []
for predictions in preds:
    predictions_softmax = np.exp(predictions) / np.sum(np.exp(predictions), axis = 2).reshape(predictions.shape[0],predictions.shape[1],1)
    predictions = predictions.argmax(-1)
    predictions_without_O = predictions_softmax[:,:,:12].argmax(-1)
    O_predictions = predictions_softmax[:,:,12]

    threshold = 0.9
    preds_final.append(np.where(O_predictions < threshold, predictions_without_O , predictions))
    
"""
---------------------------------
- eşik değeri %90 yani 0.9 olarak belirlenerek tahminler son hale getirilmiş ve "preds_final" dizisine atanır.

+ "config" değişkenine load metodu ile önceden eğitilmiş modelin yapılandırması yüklenmiş
+ yüklenen yapılandırma dosyasından etiketlerin dizisini yani "id2label" bilgilerini çekmiş ve "id2label" değişkenine atamış
+ tahminler üzerinden döngü başlatılmış
  + softmax fonksiyonu kullanılarak tahminlerin olasılıkları elde edilmiş "predictions_softmax" değişkenine atanmış
  + her bir tahmin bir sınıf indisi haline getirilmiş, en yüksek olasılığa sahip sınıfın indisi alınmış ve "predictions" değişkenine 
    atanmış
  + tahminlerde "o" etiketi içermeyen verilerin en yüksek olasılığa sahip olan etiketini belirlemiş ve "predictions_without_O" değişkenine
    atamış
  + "o" etiketlerinin olasılıklarını "O_predictions" atamış
  + eşik değeri yani güvenilirlik oranımız 0.9 olarak belirlenmiş
  + "preds_final" listesine append ile "o" etiketinin olasılığı 0.9'dan küçük olup olmamasına göre ihtimali en yüksek olan etiketi atamış
---------------------------------
"""

<div style="background-color:purple; color:white; padding:10px; border-radius: 5px; text-align:center;">
  <h1>Kod blok no: 3</h1>
</div>

### Reassembling
Note that triplets was changed to pairs to remove the FN predictions created by ignoring new triplets

In [ ]:
ds = Dataset.from_dict(ds_dict)
pairs = []
document, token, label, token_str = [], [], [], []
for p, token_map, offsets, tokens, doc in zip(preds_final, ds["token_map"], ds["offset_mapping"], ds["tokens"], ds["document"]):
    for token_pred, (start_idx, end_idx) in zip(p[0], offsets):
        label_pred = id2label[str(token_pred)]

        if start_idx + end_idx == 0: continue

        if token_map[start_idx] == -1:
            start_idx += 1

        # ignore "\n\n"
        while start_idx < len(token_map) and tokens[token_map[start_idx]].isspace():
            start_idx += 1

        if start_idx >= len(token_map): break

        token_id = token_map[start_idx]

        # ignore "O" predictions and whitespace preds
        if label_pred != "O" and token_id != -1:
            pair=(doc, token_id)

            if pair not in pairs:
                document.append(doc)
                token.append(token_id)
                label.append(label_pred)
                token_str.append(tokens[token_id])
                pairs.append(pair)
                
"""
---------------------------------
- tahminlerden ve "ds" veri kümesindeki bilgilerden yararlanarak belirli bir belirteç ile ilişkilendirilen çiftler oluşturulmuş,
  her bir çift bir tokenin tahmin edilen etiketiyle birlikte belirtilen döküman ve belirteç kimliğini içeriyor

+ Yukarıda oluşturduğumuz "ds_dict" sözlüğünden dataset oluşturulmuş "ds" değişkenine atanmış
+ çiftleri ve etiketle ilişkilendirilmiş bilgileri depolamak için boş listeler oluşturulmuş
+ tahminler ve diğer bilgiler ( token haritası, offsetler, tokenler, dökümanlar) üzerinde zip ile döngü oluşturulmuş
+ döngü içine döngü oluşturularak her bir tahmin ve offsetin eşleşmesi sağlanmış
+ "label_pred" değişkenine tahmin edilen sınıf etiketi is2label sözlüğünden atanmış
+ eğer tokenin offsetlerinin toplamı 0 ise yani token belgede yer almıyorsa döngü atlanıp sonraki tokene geçilmiş
+ tokenin başlangıcında boşluk var ise boşluk karakterini atlamak için indeks arttırılmış
+ Başlangıç indisinden başlayarak boşluk karakterlerini atlayarak geçerli bir belirteç bulana ladar döngü devam etmiş
  + indeks metnin uzunluğundan fazla olursa döngü kırılır
+ "token_id" değişkenine başlangıç indeksine karşılık gelen token map değeri atanmış
+ tahmin edilen label "o" değil ve token kimliği -1 değilse, bu durumda tahmin ve belirteç bir çift oluşturmuş ve "pair" değişkenine 
  atanmış
+ if koşulu ile de oluşturulan çiftin daha önce eklenip eklenmediği kontrol edilmiş eğer eklenmemişse listeye eklenmesi sağlanmış
---------------------------------
"""

## 🤝 Submission hand-in

In [ ]:
####
df = pd.DataFrame({
    "document": document,
    "token": token,
    "label": label,
    "token_str": token_str
})
df["row_id"] = list(range(len(df)))


"""
---------------------------------
- oluşturulan çiftlerin dataframe hali oluşturulmuş
---------------------------------
"""

# ilk 100 satırın ekrana yazdırılması
# display(df.head(100))

In [ ]:
df[["row_id", "document", "token", "label"]].to_csv("submission.csv", index=False)
"""
---------------------------------
- row_id, document, token, label değişkenleri csv formatına dönüştürülmüş
---------------------------------
"""